In [ ]:
import cv2
from pyzbar.pyzbar import decode
import sqlite3
import threading
import queue

conn = sqlite3.connect("barcode_data.db", check_same_thread=False)
cursor = conn.cursor()

cursor.execute("PRAGMA table_info(barcodes)")
columns = [col[1] for col in cursor.fetchall()]
if 'sku' not in columns:
    cursor.execute("DROP TABLE IF EXISTS barcodes")
    conn.commit()

cursor.execute('''CREATE TABLE IF NOT EXISTS barcodes (
                    sku TEXT PRIMARY KEY,
                    quantity INTEGER DEFAULT 0,
                    product_name TEXT,
                    price REAL)''')
conn.commit()

barcode_queue = queue.Queue()

def process_barcodes():
    while True:
        action, sku, product_name, price = barcode_queue.get()
        if sku == "STOP":
            break

        if action == 'add':
            cursor.execute("SELECT quantity, product_name, price FROM barcodes WHERE sku = ?", (sku,))
            result = cursor.fetchone()
            if result:
                cursor.execute("UPDATE barcodes SET quantity = quantity + 1 WHERE sku = ?", (sku,))
                conn.commit()
                print(f"Added 1 unit of SKU {sku} ({result[1]}) to the database.")
            else:
                cursor.execute("INSERT INTO barcodes (sku, quantity, product_name, price) VALUES (?, ?, ?, ?)",
                               (sku, 1, product_name, price))
                conn.commit()
                print(f"Added new product: SKU {sku} ({product_name}) to the database.")

        elif action == 'subtract':
            cursor.execute("SELECT quantity, product_name FROM barcodes WHERE sku = ?", (sku,))
            result = cursor.fetchone()
            if result and result[0] > 0:
                cursor.execute("UPDATE barcodes SET quantity = quantity - 1 WHERE sku = ?", (sku,))
                conn.commit()
                print(f"Subtracted 1 unit of SKU {sku} ({result[1]}) from the database.")

                cursor.execute("SELECT quantity FROM barcodes WHERE sku = ?", (sku,))
                if cursor.fetchone()[0] == 0:
                    cursor.execute("DELETE FROM barcodes WHERE sku = ?", (sku,))
                    conn.commit()
                    print(f"Deleted SKU {sku} ({result[1]}) from the database as quantity reached 0.")
            else:
                print(f"Cannot subtract. Either SKU {sku} does not exist or quantity is already 0.")

        print("\nCurrent Barcode Data:")
        cursor.execute("SELECT * FROM barcodes")
        rows = cursor.fetchall()
        print(f"{'SKU':<10}{'Quantity':<10}{'Product Name':<20}{'Price':<10}")
        print("-" * 50)
        for row in rows:
            sku = row[0] if row[0] is not None else "N/A"
            quantity = row[1] if row[1] is not None else 0
            product_name = row[2] if row[2] is not None else "N/A"
            price = row[3] if row[3] is not None else 0.00
            print(f"{sku:<10}{quantity:<10}{product_name:<20}{price:<10.2f}")

thread = threading.Thread(target=process_barcodes, daemon=True)
thread.start()

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

# Reduce frame resolution for faster processing
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

print("Press 'a' to add quantity, 's' to subtract quantity, 'q' to quit...")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture image.")
        break

    barcodes = decode(frame)

    for barcode in barcodes:
        x, y, w, h = barcode.rect
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        sku = barcode.data.decode('utf-8')
        text = f"SKU: {sku}"
        cv2.putText(frame, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        print(f"Detected: SKU {sku}")

    cv2.imshow('Barcode Reader', frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('a'):
        if barcodes:
            for barcode in barcodes:
                sku = barcode.data.decode('utf-8')
                cursor.execute("SELECT product_name, price FROM barcodes WHERE sku = ?", (sku,))
                result = cursor.fetchone()
                if result:
                    product_name, price = result
                    barcode_queue.put(('add', sku, product_name, price))
                else:
                    product_name = input(f"Enter product name for SKU {sku}: ")
                    price = float(input(f"Enter price for SKU {sku}: "))
                    barcode_queue.put(('add', sku, product_name, price))
        else:
            print("No barcode detected to add.")
    elif key == ord('s'):
        if barcodes:
            for barcode in barcodes:
                sku = barcode.data.decode('utf-8')
                barcode_queue.put(('subtract', sku, None, None))
        else:
            print("No barcode detected to subtract.")
    elif key == ord('q'):  # Quit
        barcode_queue.put(("STOP", None, None, None))
        break

cap.release()
cv2.destroyAllWindows()

thread.join()
conn.close()